<a href="https://colab.research.google.com/github/BernaTasel/colabWorks/blob/main/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#kütüphaneler
import collections
import io
import math
import os
import random
from six.moves import urllib

from IPython.display import clear_output, Image, display, HTML

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow_hub as hub

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics

import time

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#drive bağlantısı
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#rename the files
import os
for count, filename in enumerate(os.listdir("drive/My Drive/DietApp/PlateDataSet/non-plate/")):
  dst ="non-plate-" + str(count) + ".jpg"
  src ='drive/My Drive/DietApp/PlateDataSet/non-plate/' + filename 
  dst ='drive/My Drive/DietApp/PlateDataSet/non-plate/'+ dst
  os.rename(src, dst) 


In [6]:
#rename the files
import os
for count, filename in enumerate(os.listdir("drive/My Drive/DietApp/PlateDataSet/plate/")):
  dst ="plate-" + str(count) + ".jpg"
  src ='drive/My Drive/DietApp/PlateDataSet/plate/' + filename 
  dst ='drive/My Drive/DietApp/PlateDataSet/plate/'+ dst
  os.rename(src, dst) 


In [ ]:
#veriseti ve classları tanımlama
PLATE_DATA = 'drive/My Drive/DietApp/PlateDataSet'
def make_train_and_test_sets():
  train_examples, test_examples = [], []
  shuffler = random.Random(RANDOM_SEED)
  is_root = True
  for (dirname, subdirs, filenames) in tf.gfile.Walk(PLATE_DATA):
      if is_root:
        subdirs = sorted(subdirs)
        print(subdirs)
        lasses = collections.OrderedDict(enumerate(subdirs))
        label_to_class = dict([(x, i) for i, x in enumerate(subdirs)])
        is_root = False

In [3]:
import os
data = 'drive/My Drive/DietApp/PlateDataSet'
folders = os.listdir(data)
print(folders)

['plate', 'non-plate']
